In [1]:
import gene_expression_helper as helper
import alignment_helper as algn
import pandas as pd
import os
import numpy as np

In [2]:
input_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression"
df = pd.read_csv( os.path.join(input_directory,"section_datasets_metadata.csv"))

In [3]:
#
# First batch (Nkx2-1,Pax6,Foxp2) at (E13.5, P4, P56)
#

ff = df[df['product_name'] == 'Developing Mouse Brain']
ff = ff[[v in ['E13.5','P4','P56'] for v in ff['age']]]
ff = ff[[v in ['Nkx2-1','Pax6','Foxp2'] for v in ff['gene_symbol']]]
len(ff)

16

In [4]:
#
# Other images series from the same specimen may be useful for registration
#
specimen_list = np.unique(ff['specimen_id'])
sf = df[[v in specimen_list for v in df['specimen_id']]]
len(sf)

62

In [5]:
output_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch"
atlas_directory = "C:/Users/lydia/OneDrive/Work/DigitalAssets/atlas_models"

# make output directory
if not os.path.exists( output_directory ) :
    os.makedirs( output_directory )

In [6]:
output_file = os.path.join( output_directory, "section_datasets_metadata.csv")
sf.to_csv(output_file, index=False)

In [7]:
resample = True

for index, row in sf.iterrows() :
    
    downsample_factor = 3
    product_id = 3
    dataset_id = row['section_dataset_id']
    
    dataset_directory = os.path.join(output_directory, str(dataset_id))
    metadata_path = os.path.join(dataset_directory, "metadata_%d.json" % (dataset_id) )
    image_directory = os.path.join(dataset_directory,'section_images')
    alignment_json = os.path.join(dataset_directory, "alignment_%d.json" % (dataset_id) )
    
    print(image_directory)
    
    # make output directory
    if not os.path.exists( image_directory ) :
        os.makedirs( image_directory )
        
    helper.download_images(dataset_id,image_directory,downsample_factor)
    helper.download_metadata( dataset_id, metadata_path, product_id )
    helper.download_alignment_parameters( dataset_id, alignment_json, product_id )
    
    if resample :
        
        #
        # Use the alignment parameters to create reconstructed and registered volumes
        #
        xform = algn.read_json( alignment_json)

        # generate reconstructed volume
        output = algn.initialize_output( xform, atlas_directory, downsample_factor )
        algn.populate_output( xform, image_directory, downsample_factor, output, algn.BLUECHANNEL )

        # read atlas
        atlas = algn.read_atlas( atlas_directory, xform['age'], xform['plane_of_section'])

        # generate registered volumes
        algn.resample_to_atlas( xform, atlas, output )

        # write data to output
        resampled_directory = os.path.join( dataset_directory, 'resampled')
        os.makedirs( resampled_directory, exist_ok = True )
        algn.write_volumes( output, resampled_directory )    
        
    

C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\549\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\764\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\727\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\1144\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\68918262\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\70445269\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\70445281\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\72079959\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gene_expression/example_batch\72079884\section_images
C:/Users/lydia/OneDrive/Work/DigitalAssets/dev_mouse_gen